In [ ]:
import pandas as pd
import numpy as np
import sys, os
import matplotlib
import matplotlib.pyplot as plt
import skimage.io
# ceļu norādīt nevajag, ja pakotne ir instalēta
documentpath=os.getcwd()
libpath=os.path.abspath(os.path.join(documentpath, os.pardir))
sys.path.append(libpath)
from sudrabainiemakoni.cloudimage import CloudImage
from sudrabainiemakoni.cloudimage import Reprojector_to_Camera
from sudrabainiemakoni.cloudimage import Camera

from sudrabainiemakoni import plots
from sudrabainiemakoni import utils

In [ ]:
# katalogs ar piemēra failiem
sample_directory = 'SampleData'
# katalogs ar rezultātu failiem
results_directory = 'SampleResults'
# pirmais attēls
case1 = 'js_202206220100'
# otrais attēls
case2 = 'IMG_3916'

In [ ]:
# ielasām divus referencētus attēlus
cldim1 = CloudImage.load(f'{results_directory}/{case1}.proj')
cldim2 = CloudImage.load(f'{results_directory}/{case2}.proj')

In [ ]:
# oriģinālie attēli
fig, ax = plt.subplots(1,2, figsize=(20,10))
ax[0].imshow(cldim2.imagearray)
ax[1].imshow(cldim1.imagearray)
plt.show()

In [ ]:
# izveidojam projekcijas objektu - 1.parameters attēls, kuru projicēs, 2.parametrs - kamera ecef koordinātes -šajā gadījumā no otrā attēla
rp = Reprojector_to_Camera(cldim1, cldim2.camera.camera_ecef)

In [ ]:
# projicējam attēlu
height = 81
rp.prepare_reproject(height)
projected_image = rp.Fill_projectedImage()

In [ ]:
# uzzīmējam blakus projicētos attēlus
fig, ax = plt.subplots(1,2, figsize=(20,10))
ax[0].imshow(cldim2.imagearray)
ax[0].grid()
ax[1].imshow(projected_image)
ax[1].grid()
plt.show()

In [ ]:
#izveidojam kombinētus attēlus
img_mean, img_diff, _, img_bicolor = utils.getAverageImages([projected_image,cldim2.imagearray])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,10))
ax[0].imshow(img_diff)
ax[0].grid()
ax[1].imshow(img_bicolor[(0,1)])
ax[1].grid()
plt.show()

# Projekcija uz patvaļīgu garuma/platuma punktu un kameras skatu

In [ ]:
# testam
#az, el, rot = cldim1.camera.get_azimuth_elevation_rotation()
#lat, lon = cldim1.location.lat, cldim1.location.lon
#image_width, image_height = cldim1.imagearray.shape[1], cldim1.imagearray.shape[0]
#focallength_35mm = 24.0
# novērotāja koordinātes
lat, lon = 57.5, 27.0
# kameras skata leņķi
az, el, rot =  350, 30, 0
# kameras parametri
image_width, image_height = 1500, 1000
focallength_35mm = 24.0
# novērotāja koordinātes
#lat, lon = 59.3, 27.8
# kameras skata leņķi
#az, el, rot =  240, 40, 0
# kameras parametri
#image_width, image_height = 1500, 1000
#focallength_35mm = 24.0

In [ ]:
camera_enu, camera_ecef = Camera.make_cameras(lat,lon, 0.0, image_width, image_height, focallength_35mm, az,el,rot)

In [ ]:
rp = Reprojector_to_Camera(cldim1, camera_ecef)
cloudheight = 81
rp.prepare_reproject(cloudheight)
projected_image = rp.Fill_projectedImage()

In [ ]:
aazgrid = Camera.GetAltAzGrid_fromcamera(image_width, image_height, camera_enu)

In [ ]:
# uzzīmējam blakus projicētos attēlus
fig, ax = plt.subplots(1,2, figsize=(20,10))
ax[0].imshow(cldim1.imagearray)
ax[0].grid()
ax[1].imshow(projected_image)
plots.DrawAltAzGrid(ax[1], aazgrid)
ax[1].set_yticklabels([])
ax[1].set_xticklabels([])
ax[1].set_yticks([])
ax[1].set_xticks([])
plt.show()